In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import os
from minio import Minio
from minio.error import (ResponseError, BucketAlreadyOwnedByYou,
                         BucketAlreadyExists, BucketNotEmpty)
from k12libs.utils.nb_easy import k12ai_get_top_dir

In [4]:
minio_server='s3-internal.didiyunapi.com'
minio_access_key='AKDD002E38WR1J7RMPTGRIGNVCVINY' # os.environ.get('MINIO_ACCESS_KEY'),
minio_secret_key='ASDDXYWs45ov7MNJbj5Wc2PM9gC0FSqCIkiyQkVC' # os.environ.get('MINIO_SECRET_KEY')

bucket_name = 'talentai'
pic = os.path.join(k12ai_get_top_dir(), 'assets/bear.jpg')

In [5]:
mc = Minio(minio_server, 
           access_key=minio_access_key,
           secret_key=minio_secret_key,
           secure=True)

In [6]:
buckets = mc.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

test-talentai 2020-05-09 03:57:49+00:00
mytest 2020-05-11 02:59:32+00:00
k12ai 2020-05-11 03:40:40+00:00
talentai 2020-05-11 10:18:51+00:00


## remove_bucket

In [7]:
try: 
    mc.remove_bucket(bucket_name)
except BucketNotEmpty as err:
    print(str(err))

BucketNotEmpty: message: The bucket you tried to delete is not empty.


## make_bucket

In [8]:
try:
    mc.make_bucket(bucket_name)
except BucketAlreadyExists as err:
    print(str(err))

BucketAlreadyExists: message: The requested bucket name is not available. The bucket namespace is shared by all users of the system. Please select a different name and try again.


## list_buckets

In [9]:
buckets = mc.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

test-talentai 2020-05-09 03:57:49+00:00
mytest 2020-05-11 02:59:32+00:00
k12ai 2020-05-11 03:40:40+00:00
talentai 2020-05-11 10:18:51+00:00


## put_object

In [10]:
with open(pic, 'rb') as file_data:
    file_stat = os.stat(pic)
    print('etag:', mc.put_object(bucket_name, 'subdir/assets/bear1.jpg',
                           file_data, file_stat.st_size))

etag: 89173fd6604187189644293b8e12bcef


## fput_object

In [11]:
print(mc.fput_object(bucket_name, 'subdir/assets/bear2.jpg', pic))
print(mc.fput_object(bucket_name, 'subdir/assets/bear3.jpg', pic))

89173fd6604187189644293b8e12bcef
89173fd6604187189644293b8e12bcef


## fget_object

In [12]:
print(mc.fget_object(bucket_name, 'subdir/assets/bear2.jpg', '/tmp/bear2.jpg'))
!ls -l /tmp/bear2.jpg

<Object: bucket_name: talentai object_name: b'subdir/assets/bear2.jpg' last_modified: time.struct_time(tm_year=2020, tm_mon=5, tm_mday=12, tm_hour=14, tm_min=20, tm_sec=45, tm_wday=1, tm_yday=133, tm_isdst=0) etag: 89173fd6604187189644293b8e12bcef size: 50452 content_type: application/octet-stream, is_dir: False, metadata: {'Content-Type': 'application/octet-stream', 'Cache-Control': 'max-age=3600'}>
-rw-rw-rw- 1 root root 50452 May 12 22:20 /tmp/bear2.jpg


## list_objects

In [13]:
for obj in mc.list_objects(bucket_name, prefix='subdir/assets', recursive=True):
    print(obj)
    
print('-'*100)
    
for obj in mc.list_objects_v2(bucket_name, prefix='subdir', recursive=False):
    print(obj)

<Object: bucket_name: talentai object_name: b'subdir/assets/bear1.jpg' last_modified: 2020-05-12 14:20:45+00:00 etag: 89173fd6604187189644293b8e12bcef size: 50452 content_type: None, is_dir: False, metadata: None>
<Object: bucket_name: talentai object_name: b'subdir/assets/bear2.jpg' last_modified: 2020-05-12 14:20:45+00:00 etag: 89173fd6604187189644293b8e12bcef size: 50452 content_type: None, is_dir: False, metadata: None>
<Object: bucket_name: talentai object_name: b'subdir/assets/bear3.jpg' last_modified: 2020-05-12 14:20:45+00:00 etag: 89173fd6604187189644293b8e12bcef size: 50452 content_type: None, is_dir: False, metadata: None>
----------------------------------------------------------------------------------------------------
<Object: bucket_name: talentai object_name: b'subdir/' last_modified: None etag:  size: 0 content_type: None, is_dir: True, metadata: None>


## stat_object

In [14]:
print(mc.stat_object(bucket_name, 'subdir/assets/bear1.jpg'))
print(mc.stat_object(bucket_name, 'subdir/assets/bear2.jpg'))
# print(mc.stat_object(bucket_name, 'subdir/assets'))
for obj in mc.list_objects(bucket_name, prefix='subdir/assets/bear1.jpg', recursive=True):
    print(obj)

<Object: bucket_name: talentai object_name: b'subdir/assets/bear1.jpg' last_modified: time.struct_time(tm_year=2020, tm_mon=5, tm_mday=12, tm_hour=14, tm_min=20, tm_sec=45, tm_wday=1, tm_yday=133, tm_isdst=0) etag: 89173fd6604187189644293b8e12bcef size: 50452 content_type: application/octet-stream, is_dir: False, metadata: {'Content-Type': 'application/octet-stream', 'Cache-Control': 'max-age=3600'}>
<Object: bucket_name: talentai object_name: b'subdir/assets/bear2.jpg' last_modified: time.struct_time(tm_year=2020, tm_mon=5, tm_mday=12, tm_hour=14, tm_min=20, tm_sec=45, tm_wday=1, tm_yday=133, tm_isdst=0) etag: 89173fd6604187189644293b8e12bcef size: 50452 content_type: application/octet-stream, is_dir: False, metadata: {'Content-Type': 'application/octet-stream', 'Cache-Control': 'max-age=3600'}>
<Object: bucket_name: talentai object_name: b'subdir/assets/bear1.jpg' last_modified: 2020-05-12 14:20:45+00:00 etag: 89173fd6604187189644293b8e12bcef size: 50452 content_type: None, is_dir: F

## get_object

In [15]:
data = mc.get_object(bucket_name, 'subdir/assets/bear1.jpg')
with open('/tmp/bear.jpg', 'wb') as file_data:
    for d in data.stream():
        file_data.write(d)
        
!ls -l /tmp/bear.jpg

-rw-rw-rw- 1 root root 50452 May 12 22:20 /tmp/bear.jpg


## remove_object

In [16]:
print(mc.stat_object(bucket_name, 'subdir/assets/bear1.jpg'))
mc.remove_object(bucket_name, 'subdir/assets/bear1.jpg')

<Object: bucket_name: talentai object_name: b'subdir/assets/bear1.jpg' last_modified: time.struct_time(tm_year=2020, tm_mon=5, tm_mday=12, tm_hour=14, tm_min=20, tm_sec=45, tm_wday=1, tm_yday=133, tm_isdst=0) etag: 89173fd6604187189644293b8e12bcef size: 50452 content_type: application/octet-stream, is_dir: False, metadata: {'Content-Type': 'application/octet-stream', 'Cache-Control': 'max-age=3600'}>


In [17]:
def delete_dir(name, path):
    for obj in mc.list_objects(name, prefix=path, recursive=True):
        mc.remove_object(obj.bucket_name, obj.object_name)
delete_dir(bucket_name, 'subdir')

for obj in mc.list_objects(bucket_name, prefix='subdir', recursive=True):
    print(obj)

## Wrap Tools

In [18]:
import time
def k12ai_object_put(localpath, remotepath, prefix='framework/k12ai/'):
    with open(localpath, 'rb') as file_data:
        file_stat = os.stat(localpath)
        try:
            btime = time.time() 
            etag = mc.put_object(bucket_name, prefix + remotepath, file_data, file_stat.st_size)
            etime = time.time() 
            return etag, btime, etime, file_stat.st_size
        except ResponseError as err:
            print(str(err))
        
k12ai_object_put('/data/pretrained/cv/vgg19_bn-c79401a0.pth', '123/001/ckpts/vgg19_bn-c79401a0.pth')

('f9f2c34ab2deaf565595e7c5e0309bf9',
 1589293247.3000097,
 1589293254.4149215,
 574769405)

In [19]:
def k12ai_object_get(localpath, remotepath, prefix='framework/k12ai/'):
    try:
        return mc.fget_object(bucket_name, prefix + remotepath, localpath)
    except ResponseError as err:
        print(str(err))
        
k12ai_object_get('/tmp/cv/vgg19_bn-c79401a0.pth', '123/001/ckpts/vgg19_bn-c79401a0.pth')

KeyboardInterrupt: 

In [ ]:
def k12ai_object_rmdir(remotepath, prefix='framework/k12ai/'):
    for obj in mc.list_objects(bucket_name, prefix=prefix + remotepath, recursive=True):
        print(obj.object_name)
        mc.remove_object(obj.bucket_name, obj.object_name)
        
k12ai_object_rmdir('123/')

In [ ]:
import tarfile

def tar(fname):
    t = tarfile.open(fname + ".tar.gz", "w:gz")
    for root, dir, files in os.walk(fname):
        print(root, dir, files)
        for file in files:
            fullpath = os.path.join(root, file)
            t.add(fullpath)
    t.close()
    
tar('/data/users/26')

In [28]:
from k12libs.k12ai.k12ai_utils import k12ai_object_put

# mc.make_bucket('k12ai')

# k12ai_object_put(mc, '/data/samples/datasets.tar.gz')
# k12ai_object_put(mc, '/data/samples')
k12ai_object_put(mc, '/data/samples', prefix_map=['/data/', 'framework/'])

[{'etag': 'cbe5df6db86ec412c193495a67770771',
  'file': 'framework/samples/datasets.tar.gz',
  'size': 8075847,
  'time': [1589294087.3773189, 1589294088.3226335]},
 {'etag': 'c6dbae4a4614c3b74c93cc8d9b708e8a',
  'file': 'framework/samples/datasets/cv/KMNIST/labels_hist.png',
  'size': 12608,
  'time': [1589294088.3229141, 1589294088.3837276]},
 {'etag': '8f6bd46adbff4801196eea78ab07b520',
  'file': 'framework/samples/datasets/cv/KMNIST/imgs/test/00003.jpg',
  'size': 734,
  'time': [1589294088.3841467, 1589294088.4625175]},
 {'etag': '47be55030ecc6a1f4c437d9e2c9ee916',
  'file': 'framework/samples/datasets/cv/KMNIST/imgs/test/00043.jpg',
  'size': 670,
  'time': [1589294088.4626276, 1589294088.4979384]},
 {'etag': '88c526dce2fcd88cf5895ed96a7f3077',
  'file': 'framework/samples/datasets/cv/KMNIST/imgs/test/00001.jpg',
  'size': 630,
  'time': [1589294088.4981186, 1589294088.567009]},
 {'etag': 'c9ff7c521f82ba9a5d24473d830b4226',
  'file': 'framework/samples/datasets/cv/KMNIST/imgs/tes

In [26]:
from k12libs.k12ai.k12ai_utils import k12ai_object_get

k12ai_object_get(mc, 'framework/samples/', prefix_map=['framework', '/data/tmp'])

In [27]:
from k12libs.k12ai.k12ai_utils import k12ai_object_remove
k12ai_object_remove(mc, 'framework/samples')